# Jupyter App

In [81]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets

from epymix.rain import rain as _rain 
from epymix.inoculum import inoculum 
from epymix.configuration import configuration
from epymix.SEIR import SEIR 
from epymix.dispersion_gradient import dispersion_kernel_rust, dispersion_kernel_septo
from epymix.growth_companion import growth_pois
from epymix.SEIR_practices_data import SEIR_practice
from epymix.app_parameter_generator import parameter_set

In [82]:
#disease_par='septo_par'
delta_t0 = 10 # constant, the model has been parameterise such as t = 10 degree-day
delta_t = 10 # time step
n_season = 1 # number of season
season = 250*int(delta_t0/delta_t)  # 2500 dd %% length of a cropping season
delta_companion = 0 # growth start lag of the companion crop (dd), negative: companion before, positive: companion after but for i<delta Poi=0

Lr=1 # Lr: number of seasons
Lx=1 # Lx: number of patch along the x-axis
Ly=1 # Ly: number of patch along the y-axis
scenario_rot='uniform' # scenario_rot: rotation scenario (chose: uniform, random, chessboard, alternate, alternate_rank, etc)
wheat_fraction=0.5 # wheat_fraction: wheat fraction within each patch

In [83]:
mu_wheat = 0.03 * delta_t / delta_t0  # 0.03 %% mortality rate of S and E tissues (LAI/10dd)
nu = 0.03 * delta_t / delta_t0  # nu = mu %% mortality of I infectious tissues (LAI/10dd)
mu_companion = 0.03 * delta_t / delta_t0  # 0.03 %% mortality rate of the companion species (LAI/10dd)
beta_wheat = 0.09 * delta_t / delta_t0  # 0.09 %% wheat growth parameter (LAI/10dd)
beta_companion = 0.09 * delta_t / delta_t0  # beta_companion = beta_wheat %% growth parameter of the companion crop (LAI/10dd)
end_wheat = 140 * int(delta_t0 / delta_t)  # 1400 dj %% date of wheat growth end
end_companion = 140 * int(delta_t0 / delta_t)  # end_companion = end_wheat %% date of growth end for the companion crop
LAI_K = 6  # 6 %% carrying capacity (Maximum canopy LAI)
ber_wheat = 1  # 1 # wheat spore interception coefficient (the Beer-Lambert law)
ber_companion = 1  # 1 # spore interception coefficient of the companion species (the Beer-Lambert law)
h_wheat = 1  # wheat height
h_companion = 1  # companion height

In [84]:
year=2000

In [85]:
scenario_ino = 'initial_inoculum' # scenario_ino: which scenario of initial inoculum (chose: central_focus, initial_inoculum, annual_cloud)
frac_inf = 1 # frac_inf: fraction of patches within the field inoculated

In [86]:
### PARAMETERS IN CASE OF PRACTICE DATA
data_lai_wheat = pd.read_csv('spline_coordinate_inc_wheat_10.csv', sep=';')
data_lai_comp = pd.read_csv('spline_coordinate_inc_comp_10.csv', sep=';')
data_lai_wheat_mu = pd.read_csv('spline_coordinate_coef_wheat_10.csv', sep=';')
experiment = 5

In [87]:
# def app(crop='pure crop', disease_par='septo_par', year='2000', wheat_fraction=0.5, delta_companion = 0, view='LAIwheat'):
def app(crop='pure crop', disease_par='septo_par', experiment=5, year=2000, wheat_fraction=0.5, delta_companion = 0):
    disease, delta_ei, lambd, s0, pi_inf0, rho, psi, gamma, theta, sigma, sigma_asco, inf_begin, inoc_init_abs, ng_ext0_abs, day_length, alpha_asco, radius_asco, alpha_pycnid, radius_pycnid, alpha_ure, radius_ure = parameter_set(disease_par=disease_par, delta_t0=delta_t0, delta_t=delta_t)

    arrangement = configuration(Lr, Lx, Ly, scenario_rot, wheat_fraction)
    arrangement_max = configuration(Lr, Lx, Ly, scenario_rot, wheat_fraction=1)

    rain = _rain(year, n_season, delta_t)
    
    inoc_init, ng_ext0 = inoculum(scenario_ino, Lx, Ly, frac_inf, inoc_init_abs, ng_ext0_abs)
    
    kernel_asco, C_Disp_asco, kernel_pycnid, C_Disp_pycnid = dispersion_kernel_septo(day_length, alpha_asco, radius_asco, alpha_pycnid, radius_pycnid)
    kernel_ure, C_Disp_ure = dispersion_kernel_rust(day_length, alpha_ure, radius_ure)
    
    if crop == 'pure crop':
        t = n_season*250*int(delta_t0/delta_t) # length of the experiment
        
        Pth_inde, Poi_inde = growth_pois(t=t, season=season, arrangement=arrangement,
                    mu_companion=mu_companion, beta_companion=beta_companion, end_companion=end_companion, LAI_K=LAI_K)

        Nsp, Pth, Poi, Sth, Sus, Lat, Ifc, Ifv, Rem, LAI, LAI_wheat, Poo, Eps, AUDPC, Scont = SEIR(
        t=t, 
        delta_t0=delta_t0, delta_t=delta_t, 
        season=season, delta_companion=delta_companion,
        disease=disease, rain=rain, arrangement=arrangement, inoc_init=inoc_init, ng_ext0=ng_ext0,
        mu_wheat=mu_wheat, nu=nu, beta_wheat=beta_wheat, #beta_companion=beta_companion, end_companion=end_companion
        end_wheat=end_wheat, LAI_K=LAI_K, ber_wheat=ber_wheat, ber_companion=ber_companion, Pth_inde=Pth_inde, Poi_inde=Poi_inde,
        h_wheat=h_wheat, h_companion=h_companion,
        lambd=lambd, delta_ei=delta_ei,
        s0=s0,pi_inf0=pi_inf0, rho=rho, psi=psi, gamma=gamma, theta=theta, sigma=sigma, sigma_asco=sigma_asco, inf_begin=inf_begin,
        C_Disp_ure=C_Disp_ure, kernel_ure=kernel_ure, C_Disp_asco=C_Disp_asco, kernel_asco=kernel_asco,
        C_Disp_pycnid=C_Disp_pycnid, kernel_pycnid=kernel_pycnid)

        Nsp2, Pth2, Poi2, Sth2, Sus2, Lat2, Ifc2, Ifv2, Rem2, LAI2, LAI_wheat2, Poo2, Eps2, AUDPC2, Scont2 = SEIR(
        t=t, 
        delta_t0=delta_t0, delta_t=delta_t, 
        season=season, delta_companion=delta_companion,
        disease=disease, rain=rain, arrangement=arrangement_max, inoc_init=inoc_init, ng_ext0=ng_ext0,
        mu_wheat=mu_wheat, nu=nu, beta_wheat=beta_wheat, #beta_companion=beta_companion, end_companion=end_companion
        end_wheat=end_wheat, LAI_K=LAI_K, ber_wheat=ber_wheat, ber_companion=ber_companion, Pth_inde=Pth_inde, Poi_inde=Poi_inde,
        h_wheat=h_wheat, h_companion=h_companion,
        lambd=lambd, delta_ei=delta_ei,
        s0=s0,pi_inf0=pi_inf0, rho=rho, psi=psi, gamma=gamma, theta=theta, sigma=sigma, sigma_asco=sigma_asco, inf_begin=inf_begin,
        C_Disp_ure=C_Disp_ure, kernel_ure=kernel_ure, C_Disp_asco=C_Disp_asco, kernel_asco=kernel_asco,
        C_Disp_pycnid=C_Disp_pycnid, kernel_pycnid=kernel_pycnid)

    elif crop == 'intercropping':
        data_lai_wheat.columns.values[experiment]
        lai_wheat_inc = np.array(data_lai_wheat.iloc[:, experiment])
        lai_comp_inc = np.array(data_lai_comp.iloc[:, experiment])
        lai_wheat_mu = np.array(data_lai_wheat_mu.iloc[:, experiment])
        t = sum(~np.isnan(lai_comp_inc))
        Nsp, Pth, Poi, Sth, Sus, Lat, Ifc, Ifv, Rem, LAI, LAI_wheat, Poo, Eps, AUDPC, Scont = \
        SEIR_practice(t=t , delta_t0=delta_t0, delta_t=delta_t, season=season,
        disease=disease, rain=rain, arrangement=arrangement, inoc_init=inoc_init, ng_ext0=ng_ext0,
        lai_wheat_inc=lai_wheat_inc, lai_comp_inc=lai_comp_inc, lai_wheat_mu=lai_wheat_mu,
        nu=nu, ber_wheat=ber_wheat, ber_companion=ber_companion, h_wheat=h_wheat, h_companion=h_companion,
        lambd=lambd, delta_ei=delta_ei,
        s0=s0,pi_inf0=pi_inf0, rho=rho, psi=psi, gamma=gamma, theta=theta, sigma=sigma, sigma_asco=sigma_asco, inf_begin=inf_begin,
        C_Disp_ure=C_Disp_ure, kernel_ure=kernel_ure, C_Disp_asco=C_Disp_asco, kernel_asco=kernel_asco,
        C_Disp_pycnid=C_Disp_pycnid, kernel_pycnid=kernel_pycnid)

    
    T = [*range(0,t)]
    Sth = np.mean(Sth, axis=(1,2))
    Pth = np.mean(Pth, axis=(1,2))
    Poi = np.mean(Poi, axis=(1,2))
    Lat = np.mean(Lat, axis=(1,2))
    Ifc = np.mean(Ifc, axis=(1,2))
    Ifv = np.mean(Ifv, axis=(1,2))
    Sus = np.mean(Sus, axis=(1,2))
    Rem = np.mean(Rem, axis=(1,2))
    Nsp = np.mean(Nsp, axis=(1,2))
    LAI = np.mean(LAI, axis=(1,2))
    LAI_wheat = np.mean(LAI_wheat, axis=(1,2))
    Poo = np.mean(Poo, axis=(1,2))
    AUDPC = np.mean(AUDPC, axis=(1,2))
    Eps = np.mean(Eps, axis=(1,2))
    Scont = np.mean(Scont, axis=(1,2))
    
    # if view == 'LAIwheat':
    #     plt.plot(T, Sth, color='black')
    #     plt.ylim([0, 6])
    # elif view=='Desease':
    #     plt.plot(T, Ifc, color='red')
    #     plt.ylim([0, 6])
    # elif view=='All':
    plt.plot(T, Sth, color='black')
    plt.plot(T, Sus, color='green')
    plt.plot(T, Poi, color='brown')
    plt.plot(T, Ifv, color='red')
    plt.plot(T, Ifc, color='red',linestyle="--")
    plt.ylim([0, 6])
    plt.xlabel("Degree-day (dd.10)")
    # plt.ylabel("LAI")

    
    #print(f'AUDPC: {"{:.2f}".format(AUDPC[-1])}')
    #print(f'Disease reduction: {"{:.2f}".format(float(1-AUDPC[-1]/AUDPC2[-1]))}%')
     

In [88]:
ipywidgets.interact(app, crop=['pure crop', 'intercropping'], experiment=[('fababean_N0',61),('fababean_N1',63),('pea_N0',13),('pea_N1',14)], disease_par=['septo_par', 'rust_par'], year=list(range(1993, 2016)), wheat_fraction=(0,1,0.05), 
delta_companion=(-100,100,1))

interactive(children=(Dropdown(description='crop', options=('pure crop', 'intercropping'), value='pure crop'),…

<function __main__.app(crop='pure crop', disease_par='septo_par', experiment=5, year=2000, wheat_fraction=0.5, delta_companion=0)>